# iLykei Lecture Series

# Advanced Machine Learning and Artificial Intelligence

# Reinforcement Learning

## Notebook 6: Learning Ms. Pac-Man with DQN

## Yuri Balasanov, Mihail Tselishchev, &copy; iLykei 2018

##### Main text: Hands-On Machine Learning with Scikit-Learn and TensorFlow, Aurelien Geron, &copy; Aurelien Geron 2017, O'Reilly Media, Inc


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt


import numpy as np
import random
import time
import os
import gc

from keras.models import Sequential, clone_model
from keras.layers import Dense, Flatten, Conv2D, InputLayer
from keras.callbacks import CSVLogger, TensorBoard
from keras.optimizers import Adam
import keras.backend as K

import gym

plt.rcParams['figure.figsize'] = (9, 9)

/Users/rowena/miniconda3/envs/msca/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Deep Q-Learning of MS. Pac-Man with Keras

This notebook shows how to implement a deep neural network approach to train an agent to play Ms.Pac-Man Atari game.


## Explore the game

Use [Gym](https://gym.openai.com/) toolkit that provides both game environment and also a convenient renderer of the game.

Create an environment.

In [2]:
env = gym.make("MsPacman-ram-v0")
env.action_space  # actions are integers from 0 to 8

Discrete(9)

Try to play the game using random strategy:

In [4]:
env.reset()
done = False
score = 0
while not done:
    action = random.randrange(env.action_space.n)  # select random action
    obs, reward, done, info = env.step(action)     # make action and get results
    score += reward
    env.render()
    time.sleep(0.01)
    
env.close()
print('Score =', score)

Score = 250.0


### Observation

In this environment, observation (i.e. current state) is the RAM of the Atari machine, namely a vector of 128 bytes:

In [5]:
obs = env.reset()
print('obs shape =', obs.shape)
print('obs dtype =', obs.dtype)

obs shape = (128,)
obs dtype = uint8


Look at that vector:

In [6]:
print(obs)

[  0 112 114 115   0   3  88  88  88  88  88   0  80  80  80  50  98   0
   0   3   0   0   1   0   0   1   6   6 198   4  71   0  45   1   0 198
 198   0   0   0   0  16  52   0   0 120   0 100 130   0   0 134   1 222
   0   1   3   0   6  80 255 255   0 255 255  80 255 255  80 255 255  80
 255 255  80 191 191  80 191 191  80 191 191  80 255 255  80 255 255  80
 255 255  80 255 255   0 255 255  80 255 255  20 223  43 217 123 217 123
 217 123 217 123 217 123 217 221   0  63   0   0   0   0   0   2  66 240
 146 215]


Create a deep neural network that takes byte vector as an input and produces Q-values for state-action pairs.

## Creating a DQN-model using Keras

The following model is of the same general type applied to the cartPole problem.

Use vanilla multi-layer dense network with relu activations which computes Q-values $Q(s,a)$ for all states $s$ and actions $a$ (with some discount factor $\gamma$).
This neural network denoted by $Q(s\ |\ \theta)$ takes current state as an input and produces a vector of q-values for all 9 possible actions. Vector $\theta$ corresponds to all trainable parameters.

In [6]:
def create_dqn_model(input_shape, nb_actions, dense_layers, dense_units):
    model = Sequential()
    model.add(InputLayer(input_shape=input_shape))
    for i in range(dense_layers):
        model.add(Dense(units=dense_units, activation='relu'))
    model.add(Dense(nb_actions, activation='linear'))
    return model

Create a network using specific input shape and action space size. We call this network *online*.

In [7]:
input_shape = obs.shape
nb_actions = env.action_space.n  # 9
dense_layers = 5
dense_units = 256

online_network = create_dqn_model(input_shape, nb_actions, dense_layers, dense_units)
online_network.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_5 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_6 (Dense)              (None, 9)                 2313      
Total params: 298,505
Trainable params: 298,505
Non-trainable params: 0
_________________________________________________________________


In [8]:
from keras.utils import plot_model
plot_model(online_network, to_file='online_DenseNetwork.png',show_shapes=True,show_layer_names=True)

Plot the architecture of the network saved as *online_DenseNetwork.png*. (To see the plot log into iLykei.com, then rerun this cell).

![Model plot](https://ilykei.com/api/fileProxy/documents%2FAdvanced%20Machine%20Learning%2FReinforced%20Learning%2Fonline_DenseNetwork.png)

This network is used to explore states and rewards of Markov decision process according to an $\varepsilon$-greedy exploration strategy:

In [9]:
def epsilon_greedy(q_values, epsilon, n_outputs):
    if random.random() < epsilon:
        return random.randrange(n_outputs)  # random action
    else:
        return np.argmax(q_values)          # q-optimal action

Online network stores explored information in a *replay memory*, a double-ended queue (deque).

In [10]:
from collections import deque

replay_memory_maxlen = 1000000
replay_memory = deque([], maxlen=replay_memory_maxlen)

So, online network explores the game using $\varepsilon$-greedy strategy and saves experienced transitions in replay memory. 

In order to produce Q-values for $\varepsilon$-greedy strategy, following the proposal of the [original paper by Google DeepMind](https://www.nature.com/articles/nature14236), use another network, called *target network*, to calculate "ground-truth" target for the online network. *Target network*, has the same architecture as online network and is not going to be trained. Instead, weights from the online network are periodically copied to target network.

In [11]:
target_network = clone_model(online_network)
target_network.set_weights(online_network.get_weights())

The target network uses past experience in the form of randomly selected records of the replay memory to predict targets for the online network: 

- Select a random minibatch from replay memory containing tuples $(\text{state},\text{action},\text{reward},\text{next_state})$

- For every tuple $(\text{state},\text{action},\text{reward},\text{next_state})$ from minibatch Q-value function $Q(\text{state},\text{action}\ |\ \theta_{\text{online}})$ predicts value according to Bellman-type equation: 

$$y_\text{target} = \text{reward} + \gamma \cdot \max_a Q(\text{next_state}, a\ |\ \theta_\text{target})$$
if the game continues and $$ y_\text{target} = \text{reward}$$ if the game has ended. 

Note that at this step predictions are made by the target network. This helps preventing situations when online network simultaneously predicts values and creates targets, which might potentially lead to instability of training process.

- For each record in the minibatch targets need to be calculated for only one specific $\text{action}$ output of online network. It is important to ignore all other outputs during optimization (calculating gradients). So, predictions for every record in the minibatch are calculated by online network first, then the values corresponding to the taction of interest are replaced with ones predicted by target network. 

Proposed approach is called **DQN**-approach. However, [it is known](https://arxiv.org/abs/1509.06461) to overestimate q-values under certain conditions. In the paper another approach was proposed, called **Double DQN**. Instead of taking action that maximizes q-value for target network, they pick an action that maximizes q-value for online network as an optimal one:

$$y_\text{target} = \text{reward} + \gamma \cdot Q\left(\text{next_state}, \arg\max_a Q\left(\text{next_state},a\ |\ \theta_\text{online}\right)\ |\ \theta_\text{target}\right).$$


## Training DQN

First, define hyperparameters (Do not forget to change them before moving to cluster):

In [12]:
name = 'MsPacman_DQN'  # used in naming files (weights, logs, etc)
n_steps = 10000        # total number of training steps (= n_epochs)
warmup = 1000          # start training after warmup iterations
training_interval = 4  # period (in actions) between training steps
save_steps = int(n_steps/10)  # period (in training steps) between storing weights to file
copy_steps = 100       # period (in training steps) between updating target_network weights
gamma = 0.9            # discount rate
skip_start = 90        # skip the start of every game (it's just freezing time before game starts)
batch_size = 64        # size of minibatch that is taken randomly from replay memory every training step
double_dqn = False     # whether to use Double-DQN approach or simple DQN (see above)
# eps-greedy parameters: we slowly decrease epsilon from eps_max to eps_min in eps_decay_steps
eps_max = 1.0
eps_min = 0.05
eps_decay_steps = int(n_steps/2)

learning_rate = 0.001

Compile online-network with Adam optimizer, mean squared error loss and `mean_q` metric, which measures the maximum of predicted q-values averaged over samples from minibatch (we expect it to increase during training process).

In [13]:
def mean_q(y_true, y_pred):
    return K.mean(K.max(y_pred, axis=-1))


online_network.compile(optimizer=Adam(learning_rate), loss='mse', metrics=[mean_q])

Create folder for logs and trained weights:

In [14]:
if not os.path.exists(name):
    os.makedirs(name)
    
weights_folder = os.path.join(name, 'weights')
if not os.path.exists(weights_folder):
    os.makedirs(weights_folder)

Use standard callbacks:

In [15]:
csv_logger = CSVLogger(os.path.join(name, 'log.csv'), append=True, separator=';')
tensorboard = TensorBoard(log_dir=os.path.join(name, 'tensorboard'), write_graph=False, write_images=False)

/home/yuri/anaconda3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Next chunk of code explores the game, trains online network and periodically copies weights to target network as explained above.

In [16]:
# counters:
step = 0          # training step counter (= epoch counter)
iteration = 0     # frames counter
episodes = 0      # game episodes counter
done = True       # indicator that env needs to be reset

episode_scores = []  # collect total scores in this list and log it later

while step < n_steps:
    if done:  # game over, restart it
        obs = env.reset()
        score = 0  # reset score for current episode
        for skip in range(skip_start):  # skip the start of each game (it's just freezing time before game starts)
            obs, reward, done, info = env.step(0)
            score += reward
        state = obs
        episodes += 1

    # Online network evaluates what to do
    iteration += 1
    q_values = online_network.predict(np.array([state]))[0]  # calculate q-values using online network
    # select epsilon (which linearly decreases over training steps):
    epsilon = max(eps_min, eps_max - (eps_max-eps_min) * step/eps_decay_steps)
    action = epsilon_greedy(q_values, epsilon, nb_actions)
    # Play:
    obs, reward, done, info = env.step(action)
    score += reward
    if done:
        episode_scores.append(score)
    next_state = obs
    # Let's memorize what just happened
    replay_memory.append((state, action, reward, next_state, done))
    state = next_state

    if iteration >= warmup and iteration % training_interval == 0:
        # learning branch
        step += 1
        minibatch = random.sample(replay_memory, batch_size)
        replay_state = np.array([x[0] for x in minibatch])
        replay_action = np.array([x[1] for x in minibatch])
        replay_rewards = np.array([x[2] for x in minibatch])
        replay_next_state = np.array([x[3] for x in minibatch])
        replay_done = np.array([x[4] for x in minibatch], dtype=int)

        # calculate targets (see above for details)
        if double_dqn == False:
            # DQN
            target_for_action = replay_rewards + (1-replay_done) * gamma * \
                                    np.amax(target_network.predict(replay_next_state), axis=1)
        else:
            # Double DQN
            best_actions = np.argmax(online_network.predict(replay_next_state), axis=1)
            target_for_action = replay_rewards + (1-replay_done) * gamma * \
                                    target_network.predict(replay_next_state)[np.arange(batch_size), best_actions]

        target = online_network.predict(replay_state)  # targets coincide with predictions ...
        target[np.arange(batch_size), replay_action] = target_for_action  #...except for targets with actions from replay
        
        # Train online network
        online_network.fit(replay_state, target, epochs=step, verbose=2, initial_epoch=step-1,
                           callbacks=[csv_logger, tensorboard])

        # Periodically copy online network weights to target network
        if step % copy_steps == 0:
            target_network.set_weights(online_network.get_weights())
        # And save weights
        if step % save_steps == 0:
            online_network.save_weights(os.path.join(weights_folder, 'weights_{}.h5f'.format(step)))
            gc.collect()  # also clean the garbage

Epoch 1/1
 - 0s - loss: 1367.4331 - mean_q: 55.3535
Epoch 2/2
 - 0s - loss: 298.6378 - mean_q: 43.8116
Epoch 3/3
 - 0s - loss: 200.6864 - mean_q: 42.5586
Epoch 4/4
 - 0s - loss: 139.5677 - mean_q: 41.9298
Epoch 5/5
 - 0s - loss: 89.5148 - mean_q: 43.1367
Epoch 6/6
 - 0s - loss: 77.4191 - mean_q: 45.0979
Epoch 7/7
 - 0s - loss: 24.3183 - mean_q: 44.8210
Epoch 8/8
 - 0s - loss: 14.7073 - mean_q: 44.1121
Epoch 9/9
 - 0s - loss: 15.0842 - mean_q: 44.6151
Epoch 10/10
 - 0s - loss: 8.5063 - mean_q: 45.3018
Epoch 11/11
 - 0s - loss: 6.3824 - mean_q: 46.1520
Epoch 12/12
 - 0s - loss: 6.0075 - mean_q: 47.0970
Epoch 13/13
 - 0s - loss: 6.0216 - mean_q: 46.7011
Epoch 14/14
 - 0s - loss: 4.6321 - mean_q: 46.5515
Epoch 15/15
 - 0s - loss: 3.8754 - mean_q: 48.0801
Epoch 16/16
 - 0s - loss: 3.3106 - mean_q: 47.7287
Epoch 17/17
 - 0s - loss: 4.5783 - mean_q: 48.4402
Epoch 18/18
 - 0s - loss: 2.8497 - mean_q: 46.7193
Epoch 19/19
 - 0s - loss: 3.5541 - mean_q: 47.6507
Epoch 20/20
 - 0s - loss: 2.4385 - 

Epoch 160/160
 - 0s - loss: 0.8102 - mean_q: 43.8597
Epoch 161/161
 - 0s - loss: 0.6748 - mean_q: 43.0889
Epoch 162/162
 - 0s - loss: 0.6433 - mean_q: 43.7292
Epoch 163/163
 - 0s - loss: 2.3679 - mean_q: 43.1206
Epoch 164/164
 - 0s - loss: 0.6344 - mean_q: 43.7586
Epoch 165/165
 - 0s - loss: 0.6882 - mean_q: 43.7300
Epoch 166/166
 - 0s - loss: 2.7072 - mean_q: 43.5952
Epoch 167/167
 - 0s - loss: 0.7895 - mean_q: 42.4912
Epoch 168/168
 - 0s - loss: 0.8277 - mean_q: 43.4678
Epoch 169/169
 - 0s - loss: 0.8312 - mean_q: 43.2379
Epoch 170/170
 - 0s - loss: 1.0317 - mean_q: 44.7064
Epoch 171/171
 - 0s - loss: 1.4617 - mean_q: 42.3833
Epoch 172/172
 - 0s - loss: 1.6365 - mean_q: 44.8247
Epoch 173/173
 - 0s - loss: 4.1249 - mean_q: 42.5808
Epoch 174/174
 - 0s - loss: 2.4476 - mean_q: 43.9347
Epoch 175/175
 - 0s - loss: 1.7684 - mean_q: 42.5688
Epoch 176/176
 - 0s - loss: 3.0153 - mean_q: 44.1157
Epoch 177/177
 - 0s - loss: 4.1158 - mean_q: 41.9139
Epoch 178/178
 - 0s - loss: 2.2741 - mean_q: 4

Epoch 315/315
 - 0s - loss: 0.2266 - mean_q: 38.0562
Epoch 316/316
 - 0s - loss: 0.4058 - mean_q: 38.4356
Epoch 317/317
 - 0s - loss: 0.8935 - mean_q: 38.4827
Epoch 318/318
 - 0s - loss: 1.5022 - mean_q: 38.7863
Epoch 319/319
 - 0s - loss: 0.4039 - mean_q: 38.7820
Epoch 320/320
 - 0s - loss: 0.5392 - mean_q: 38.3393
Epoch 321/321
 - 0s - loss: 0.6715 - mean_q: 38.5434
Epoch 322/322
 - 0s - loss: 0.8062 - mean_q: 38.1220
Epoch 323/323
 - 0s - loss: 0.5467 - mean_q: 39.1151
Epoch 324/324
 - 0s - loss: 0.6089 - mean_q: 38.4309
Epoch 325/325
 - 0s - loss: 0.4453 - mean_q: 38.1297
Epoch 326/326
 - 0s - loss: 0.7246 - mean_q: 38.5898
Epoch 327/327
 - 0s - loss: 0.5583 - mean_q: 38.5208
Epoch 328/328
 - 0s - loss: 1.0584 - mean_q: 38.5384
Epoch 329/329
 - 0s - loss: 0.4698 - mean_q: 39.1113
Epoch 330/330
 - 0s - loss: 0.5808 - mean_q: 38.4325
Epoch 331/331
 - 0s - loss: 0.5567 - mean_q: 38.7890
Epoch 332/332
 - 0s - loss: 2.0756 - mean_q: 38.2618
Epoch 333/333
 - 0s - loss: 0.2098 - mean_q: 3

Epoch 470/470
 - 0s - loss: 0.9062 - mean_q: 35.9486
Epoch 471/471
 - 0s - loss: 0.5487 - mean_q: 35.7162
Epoch 472/472
 - 0s - loss: 0.9596 - mean_q: 36.2709
Epoch 473/473
 - 0s - loss: 0.7968 - mean_q: 36.0744
Epoch 474/474
 - 0s - loss: 0.8124 - mean_q: 36.8016
Epoch 475/475
 - 0s - loss: 2.2150 - mean_q: 35.9118
Epoch 476/476
 - 0s - loss: 0.7879 - mean_q: 36.7423
Epoch 477/477
 - 0s - loss: 0.9651 - mean_q: 36.0697
Epoch 478/478
 - 0s - loss: 0.8281 - mean_q: 36.7736
Epoch 479/479
 - 0s - loss: 0.7327 - mean_q: 35.5387
Epoch 480/480
 - 0s - loss: 0.7448 - mean_q: 36.2403
Epoch 481/481
 - 0s - loss: 1.3755 - mean_q: 35.5025
Epoch 482/482
 - 0s - loss: 1.2268 - mean_q: 36.6466
Epoch 483/483
 - 0s - loss: 1.2917 - mean_q: 35.0490
Epoch 484/484
 - 0s - loss: 1.1733 - mean_q: 36.2243
Epoch 485/485
 - 0s - loss: 1.1694 - mean_q: 35.1263
Epoch 486/486
 - 0s - loss: 1.1687 - mean_q: 36.9601
Epoch 487/487
 - 0s - loss: 1.2286 - mean_q: 35.4705
Epoch 488/488
 - 0s - loss: 1.0709 - mean_q: 3

Epoch 625/625
 - 0s - loss: 0.1691 - mean_q: 32.6806
Epoch 626/626
 - 0s - loss: 0.8342 - mean_q: 31.7829
Epoch 627/627
 - 0s - loss: 0.1294 - mean_q: 32.3910
Epoch 628/628
 - 0s - loss: 1.1576 - mean_q: 32.0124
Epoch 629/629
 - 0s - loss: 0.6522 - mean_q: 32.7086
Epoch 630/630
 - 0s - loss: 1.0465 - mean_q: 32.6995
Epoch 631/631
 - 0s - loss: 0.5096 - mean_q: 31.9637
Epoch 632/632
 - 0s - loss: 0.5980 - mean_q: 32.8139
Epoch 633/633
 - 0s - loss: 0.5308 - mean_q: 32.3482
Epoch 634/634
 - 0s - loss: 0.2352 - mean_q: 32.3389
Epoch 635/635
 - 0s - loss: 0.5934 - mean_q: 32.0473
Epoch 636/636
 - 0s - loss: 0.7928 - mean_q: 32.4116
Epoch 637/637
 - 0s - loss: 0.7101 - mean_q: 32.4047
Epoch 638/638
 - 0s - loss: 0.3269 - mean_q: 31.6222
Epoch 639/639
 - 0s - loss: 0.6586 - mean_q: 32.4282
Epoch 640/640
 - 0s - loss: 0.5477 - mean_q: 32.5281
Epoch 641/641
 - 0s - loss: 0.4476 - mean_q: 32.3793
Epoch 642/642
 - 0s - loss: 0.7028 - mean_q: 32.1135
Epoch 643/643
 - 0s - loss: 0.5915 - mean_q: 3

...

Epoch 9279/9279
 - 0s - loss: 0.5549 - mean_q: 15.4200
Epoch 9280/9280
 - 0s - loss: 0.7839 - mean_q: 15.8102
Epoch 9281/9281
 - 0s - loss: 0.6348 - mean_q: 15.9283
Epoch 9282/9282
 - 0s - loss: 0.4388 - mean_q: 15.5873
Epoch 9283/9283
 - 0s - loss: 1.0886 - mean_q: 16.3470
Epoch 9284/9284
 - 0s - loss: 0.4359 - mean_q: 15.9547
Epoch 9285/9285
 - 0s - loss: 0.5658 - mean_q: 15.3626
Epoch 9286/9286
 - 0s - loss: 0.2689 - mean_q: 16.4019
Epoch 9287/9287
 - 0s - loss: 0.7914 - mean_q: 14.9664
Epoch 9288/9288
 - 0s - loss: 16.1350 - mean_q: 16.1262
Epoch 9289/9289
 - 0s - loss: 1.2163 - mean_q: 15.7442
Epoch 9290/9290
 - 0s - loss: 0.4411 - mean_q: 16.2541
Epoch 9291/9291
 - 0s - loss: 0.7107 - mean_q: 15.7653
Epoch 9292/9292
 - 0s - loss: 0.3642 - mean_q: 16.1402
Epoch 9293/9293
 - 0s - loss: 0.5955 - mean_q: 15.0779
Epoch 9294/9294
 - 0s - loss: 0.6964 - mean_q: 16.8717
Epoch 9295/9295
 - 0s - loss: 0.8859 - mean_q: 14.9493
Epoch 9296/9296
 - 0s - loss: 0.4847 - mean_q: 16.3278
Epoch 929

Epoch 9428/9428
 - 0s - loss: 0.4207 - mean_q: 13.6439
Epoch 9429/9429
 - 0s - loss: 270.1959 - mean_q: 16.4361
Epoch 9430/9430
 - 0s - loss: 2.0210 - mean_q: 14.6493
Epoch 9431/9431
 - 0s - loss: 1.4235 - mean_q: 15.7674
Epoch 9432/9432
 - 0s - loss: 1.1188 - mean_q: 16.7256
Epoch 9433/9433
 - 0s - loss: 1.0620 - mean_q: 17.5018
Epoch 9434/9434
 - 0s - loss: 1.2471 - mean_q: 18.5242
Epoch 9435/9435
 - 0s - loss: 0.7129 - mean_q: 18.8198
Epoch 9436/9436
 - 0s - loss: 0.8242 - mean_q: 18.0034
Epoch 9437/9437
 - 0s - loss: 0.6759 - mean_q: 17.6898
Epoch 9438/9438
 - 0s - loss: 0.7748 - mean_q: 18.2341
Epoch 9439/9439
 - 0s - loss: 0.8644 - mean_q: 18.4423
Epoch 9440/9440
 - 0s - loss: 1.1077 - mean_q: 17.2129
Epoch 9441/9441
 - 0s - loss: 0.9873 - mean_q: 17.8272
Epoch 9442/9442
 - 0s - loss: 0.5325 - mean_q: 17.5248
Epoch 9443/9443
 - 0s - loss: 0.4970 - mean_q: 16.9496
Epoch 9444/9444
 - 0s - loss: 0.4898 - mean_q: 16.4958
Epoch 9445/9445
 - 0s - loss: 0.7642 - mean_q: 17.4917
Epoch 94

Epoch 9577/9577
 - 0s - loss: 0.8145 - mean_q: 16.4027
Epoch 9578/9578
 - 0s - loss: 0.3078 - mean_q: 16.0661
Epoch 9579/9579
 - 0s - loss: 0.4599 - mean_q: 15.7700
Epoch 9580/9580
 - 0s - loss: 0.6847 - mean_q: 15.2798
Epoch 9581/9581
 - 0s - loss: 0.4734 - mean_q: 15.7866
Epoch 9582/9582
 - 0s - loss: 0.5479 - mean_q: 16.0426
Epoch 9583/9583
 - 0s - loss: 0.2851 - mean_q: 16.9749
Epoch 9584/9584
 - 0s - loss: 0.1971 - mean_q: 15.7770
Epoch 9585/9585
 - 0s - loss: 0.3284 - mean_q: 15.4324
Epoch 9586/9586
 - 0s - loss: 0.6868 - mean_q: 15.8736
Epoch 9587/9587
 - 0s - loss: 0.4408 - mean_q: 16.7597
Epoch 9588/9588
 - 0s - loss: 0.2567 - mean_q: 15.0434
Epoch 9589/9589
 - 0s - loss: 0.6367 - mean_q: 15.7448
Epoch 9590/9590
 - 0s - loss: 64.3678 - mean_q: 16.6199
Epoch 9591/9591
 - 0s - loss: 0.6111 - mean_q: 15.9568
Epoch 9592/9592
 - 0s - loss: 6.6489 - mean_q: 16.8735
Epoch 9593/9593
 - 0s - loss: 0.7532 - mean_q: 17.3715
Epoch 9594/9594
 - 0s - loss: 0.6785 - mean_q: 17.4364
Epoch 959

Epoch 9726/9726
 - 0s - loss: 0.2845 - mean_q: 15.8431
Epoch 9727/9727
 - 0s - loss: 0.3783 - mean_q: 14.4182
Epoch 9728/9728
 - 0s - loss: 0.4870 - mean_q: 15.2510
Epoch 9729/9729
 - 0s - loss: 0.3488 - mean_q: 15.4214
Epoch 9730/9730
 - 0s - loss: 0.5609 - mean_q: 14.6229
Epoch 9731/9731
 - 0s - loss: 1.2172 - mean_q: 15.5958
Epoch 9732/9732
 - 0s - loss: 0.7042 - mean_q: 16.2657
Epoch 9733/9733
 - 0s - loss: 0.6957 - mean_q: 16.2042
Epoch 9734/9734
 - 0s - loss: 0.5733 - mean_q: 15.0976
Epoch 9735/9735
 - 0s - loss: 0.4488 - mean_q: 15.5235
Epoch 9736/9736
 - 0s - loss: 0.6289 - mean_q: 15.7102
Epoch 9737/9737
 - 0s - loss: 0.2295 - mean_q: 16.8290
Epoch 9738/9738
 - 0s - loss: 0.3959 - mean_q: 15.2086
Epoch 9739/9739
 - 0s - loss: 6.2445 - mean_q: 15.8904
Epoch 9740/9740
 - 0s - loss: 0.3992 - mean_q: 16.0908
Epoch 9741/9741
 - 0s - loss: 0.2378 - mean_q: 15.2790
Epoch 9742/9742
 - 0s - loss: 0.4078 - mean_q: 15.1420
Epoch 9743/9743
 - 0s - loss: 0.4076 - mean_q: 15.6373
Epoch 9744

Epoch 9875/9875
 - 0s - loss: 0.4849 - mean_q: 16.4848
Epoch 9876/9876
 - 0s - loss: 0.4990 - mean_q: 17.4153
Epoch 9877/9877
 - 0s - loss: 0.3788 - mean_q: 16.5704
Epoch 9878/9878
 - 0s - loss: 0.4660 - mean_q: 15.8362
Epoch 9879/9879
 - 0s - loss: 0.4192 - mean_q: 16.6716
Epoch 9880/9880
 - 0s - loss: 0.5712 - mean_q: 16.4049
Epoch 9881/9881
 - 0s - loss: 0.1808 - mean_q: 15.4896
Epoch 9882/9882
 - 0s - loss: 0.7074 - mean_q: 14.8296
Epoch 9883/9883
 - 0s - loss: 0.6124 - mean_q: 16.5078
Epoch 9884/9884
 - 0s - loss: 0.3432 - mean_q: 16.1278
Epoch 9885/9885
 - 0s - loss: 0.4824 - mean_q: 15.2797
Epoch 9886/9886
 - 0s - loss: 0.4270 - mean_q: 13.6879
Epoch 9887/9887
 - 0s - loss: 0.5468 - mean_q: 14.5375
Epoch 9888/9888
 - 0s - loss: 0.5234 - mean_q: 16.2222
Epoch 9889/9889
 - 0s - loss: 0.7230 - mean_q: 16.6361
Epoch 9890/9890
 - 0s - loss: 0.5635 - mean_q: 15.0196
Epoch 9891/9891
 - 0s - loss: 0.5019 - mean_q: 16.6150
Epoch 9892/9892
 - 0s - loss: 0.3996 - mean_q: 14.9164
Epoch 9893

Save last weights:

In [17]:
online_network.save_weights(os.path.join(weights_folder, 'weights_last.h5f'))

In [18]:
# Dump all scores to txt-file
with open(os.path.join(name, 'episode_scores.txt'), 'w') as file:
    for item in episode_scores:
        file.write("{}\n".format(item))

print(episode_scores)

[190.0, 230.0, 210.0, 180.0, 210.0, 350.0, 290.0, 580.0, 220.0, 140.0, 210.0, 290.0, 260.0, 340.0, 220.0, 140.0, 630.0, 290.0, 670.0, 490.0, 260.0, 340.0, 1320.0, 290.0, 370.0, 360.0, 300.0, 380.0, 270.0, 880.0, 600.0, 210.0, 580.0, 370.0, 640.0, 1120.0, 410.0, 190.0, 150.0, 520.0, 190.0, 150.0, 330.0, 220.0, 280.0, 360.0, 420.0, 300.0, 700.0, 290.0, 280.0, 170.0, 730.0, 1150.0, 700.0, 280.0, 310.0, 570.0, 270.0, 530.0, 1030.0, 950.0, 210.0, 250.0, 2540.0, 120.0, 750.0, 390.0, 250.0, 390.0]


Don't forget to check TensorBoard for fancy statistics on loss and metrics using

*tensorboard --logdir=tensorboard*

and visit http://localhost:6006/

## Testing model

Finally, create a function to evalutate the trained network. 
Note that we still using $\varepsilon$-greedy strategy here to prevent an agent from getting stuck. 
`test_dqn` returns a list with scores for specific number of games.

In [19]:
def test_dqn(env, n_games, model, nb_actions, skip_start, eps=0.05, render=False, sleep_time=0.01):
    scores = []
    for i in range(n_games):
        obs = env.reset()
        score = 0
        done = False
        for skip in range(skip_start):  # skip the start of each game (it's just freezing time before game starts)
            obs, reward, done, info = env.step(0)
            score += reward
        while not done:
            state = obs
            q_values = model.predict(np.array([state]))[0]
            action = epsilon_greedy(q_values, eps, nb_actions)
            obs, reward, done, info = env.step(action)
            score += reward
            if render:
                env.render()
                time.sleep(sleep_time)
                if done:
                    time.sleep(1)
        scores.append(score)
    return scores

In [21]:
ngames = 5
scores = test_dqn(env, ngames, online_network, nb_actions, skip_start, render=True)
print(scores)

[130.0, 60.0, 110.0, 110.0, 60.0]


In [22]:
env.close()

Results are pretty poor since the training was too short. 

Try to train DQN on a cluster. You might want to adjust some hyperparameters (increase `n_steps`, `warmup`, `copy_steps` and `eps_decay_steps`; gradually decrease learning rate during training, select appropriate `batch_size` to fit gpu memory, adjust `gamma`, switch on double dqn apporach and so on). 

You can even try to make the network deeper and/or use more than one observation as an input of neural network. For instance, using few consecutive game observations would definetely improve the results since they contain some helpful information such as monsters directions, etc. Turning off TensorBoard callback on a cluster would be a good idea too.